- Gabriel Quiroz 19255
- Jose Pablo Ponce 19092

In [2]:
#Importar las librerías

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

### Exploración de datos

1. Cargue el dataset en un dataframe de pandas, muestre un ejemplo de cinco observaciones.


In [80]:
## Cargar el dataset proporcionado
df = pd.read_csv('dataset_pishing.csv')
print(df.shape)
pd.set_option('display.max_columns', None)
df.sample(n=5).head() 

(11430, 67)


,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
3338,http://support-appleld.com.secureupdate.duilaw...,1,0,1,0,0,0,0,0,1,0,3,1,0,0,0,1,0,13,2,2,3,2,17,13,17,7.769231,8.400000,7.375000,0,0,0,0,0,0,4,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.000000,0.000000,0,0,0,100.0,0,0,0,1,0,0,14,4003,5816617,0,1,0,phishing
7309,https://www.volley.de/,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,4,3,3,0,6,6,0,4.500000,4.500000,0.000000,0,0,0,0,0,0,61,0.639344,0.360656,0,0,0,0.409091,0,0.000000,0,0,87.500000,0,64.285714,35.714286,0,0,0,0.0,0,0,0,0,1,0,0,-1,5750434,0,0,3,legitimate
6999,https://tradepointonline-my.sharepoint.com/:o:...,0,0,1,0,0,0,0,1,0,0,2,1,0,1,0,1,0,15,2,1,2,1,46,16,46,7.533333,9.333333,7.083333,0,0,0,0,0,0,7,0.000000,1.000000,0,0,0,0.000000,0,0.571429,0,1,0.000000,0,0.000000,0.000000,0,0,0,0.0,0,0,0,1,0,0,382,8018,0,0,1,4,phishing
1545,http://www.kidscoloringpdf.com/wp-content/xxx/...,0,1,0,0,0,0,0,0,0,0,3,1,0,0,0,1,0,14,8,2,3,2,15,15,9,6.142857,9.000000,5.666667,2,0,0,0,0,0,4,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.000000,0.000000,0,0,0,100.0,0,0,0,1,0,0,395,1797,0,0,1,0,phishing
1562,http://www.get-piano-lessons.com/layout-piano-...,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,8,4,3,3,4,7,7,6,4.625000,4.500000,4.750000,0,0,0,0,0,0,29,0.724138,0.275862,0,0,0,0.125000,0,0.000000,1,0,45.454545,0,0.000000,100.000000,0,0,0,0.0,0,0,0,1,1,0,227,4887,0,0,0,3,legitimate


2. Muestre la cantidad de observaciones etiquetadas en la columna status como “legit” y como
“pishing”. ¿Está balanceado el dataset?

In [81]:
print('legitimate:', len(df[df['status']=='legitimate']))
print('phising:', len(df[df['status']=='phishing']))

legitimate: 5715
phising: 5715


Como se puede observar hay la misma cantidad de observaciones tanto para "legitimate" y "phising", por ende se puede afirmar que el dataset esta balanceado

### Derivación de características


En base al artículo “Towards Benchmark Datasets for ML Based Wensite Phishing Detection: An
Experimental Study”, derivar las características basadas en el dominio: f1, f2, f4 – f20, f25, f26 y f27.

In [82]:
from urllib.parse import urlparse
def get_hostname(url):
    o = urlparse(url)
    return o.hostname
    
df['hostname'] = df['url'].apply(get_hostname)


In [83]:
#f1
df['f1'] = df['url'].str.len()


In [84]:

#f2
df['f2'] = df['hostname'].str.len()


In [85]:
#f4
df['f4'] = df['url'].str.count('\\.')

#f5
df['f5'] = df['url'].str.count('\\-')

#f6
df['f6'] = df['url'].str.count('\\@')

#f7
df['f7'] = df['url'].str.count('\\?')

#f8
df['f8'] = df['url'].str.count('\\&')

#f9
df['f9'] = df['url'].str.count('\\|')

#f10
df['f10'] = df['url'].str.count('\\=')

#f11
df['f11'] = df['url'].str.count('\\_')

#f12
df['f12'] = df['url'].str.count('\\˜')

#f13
df['f13'] = df['url'].str.count('\\%')

#f14
df['f14'] = df['url'].str.count('\\/')

#f15
df['f15'] = df['url'].str.count('\\*')

#f16
df['f16'] = df['url'].str.count('\\:')

#f17
df['f17'] = df['url'].str.count('\\,')

#f18
df['f18'] = df['url'].str.count('\\;')

#f19
df['f19'] = df['url'].str.count('\\$')

#f20
df['f20'] = df['url'].str.count('\\%20')


In [86]:
#f25
df['f25'] = df['url'].str.startswith('https')
df["f25"] = df["f25"].astype(int)


In [87]:
#f26
def count_digits(url):
    digits = sum(c.isdigit() for c in url)
    url_len = len(url)
    return digits / url_len

df["f26"] = df["url"].apply(count_digits)

In [88]:
#f27
df["f27"] = df["hostname"].apply(count_digits)

In [92]:
df.sample(n=5).head() 

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,abnormal_subdomain,nb_subdomains,prefix_suffix,random_domain,shortening_service,path_extension,nb_redirection,nb_external_redirection,length_words_raw,char_repeat,shortest_words_raw,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path,phish_hints,domain_in_brand,brand_in_subdomain,brand_in_path,suspecious_tld,statistical_report,nb_hyperlinks,ratio_intHyperlinks,ratio_extHyperlinks,ratio_nullHyperlinks,nb_extCSS,ratio_intRedirection,ratio_extRedirection,ratio_intErrors,ratio_extErrors,login_form,external_favicon,links_in_tags,submit_email,ratio_intMedia,ratio_extMedia,sfh,iframe,popup_window,safe_anchor,onmouseover,right_clic,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status,hostname,f1,f2,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f25,f26,f27
7145,http://bioenergiasilesia.pl/mail/sign-login-pa...,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,11,1,2,17,2,17,17,8,5.727273,17.0,4.6,2,0,0,0,0,0,23,0.826087,0.173913,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.0,0.0,0,0,0,66.666667,0,0,0,1,1,0,46,3242,0,0,1,2,phishing,bioenergiasilesia.pl,84,20,3,5,0,0,0,0,0,0,0,0,6,0,1,0,0,0,0,0,0.0,0.0
8259,https://eddimonster.com/wordp/theteam/planis/s...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,6,2,3,11,3,11,11,10,7.000000,11.0,6.2,0,0,0,0,0,0,3,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,100.000000,0,0.0,0.0,0,0,0,0.000000,0,0,0,1,0,0,305,60,7866174,0,1,0,phishing,eddimonster.com,60,15,1,0,0,0,0,0,0,0,0,0,8,0,1,0,0,0,0,1,0.0,0.0
731,https://www.alaskausa.org/verify/update,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,4,3,3,3,6,9,9,6,6.000000,6.0,6.0,0,0,0,0,0,0,215,0.972093,0.027907,0,0,0,0.166667,0,0.166667,0,0,90.909091,0,75.0,25.0,0,0,0,91.111111,0,0,0,1,1,0,1228,8632,49662,0,1,4,phishing,www.alaskausa.org,39,17,2,0,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,1,0.0,0.0
751,http://designfitters.com/poster/adobe,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,3,1,5,13,5,13,13,6,8.000000,13.0,5.5,0,0,0,0,0,0,10,1.000000,0.000000,0,0,0,0.000000,0,0.000000,0,0,0.000000,0,100.0,0.0,0,0,0,100.000000,0,0,0,1,1,0,187,909,0,0,1,2,phishing,designfitters.com,37,17,1,0,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,0,0.0,0.0
2661,https://en.wiktionary.org/wiki/corpus,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,0,2,2,4,10,10,6,5.500000,6.0,5.0,0,0,0,0,0,0,725,0.899310,0.100690,0,0,0,0.219178,0,0.000000,0,0,100.000000,0,100.0,0.0,0,0,0,74.740484,0,0,0,0,1,0,509,6431,623,0,0,5,legitimate,en.wiktionary.org,37,17,2,0,0,0,0,0,0,0,0,0,4,0,1,0,0,0,0,1,0.0,0.0
